In [ ]:
import torch
import pandas as pd
from IPython.display import display
from models.transfer_models import get_resnet18
from models.custom_cnn import CustomCNN
from utils.evaluate import evaluate   # якщо у тебе є така функція
from utils.misc import model_size     # якщо у тебе є така функція
from torch.utils.data import DataLoader
from utils.dataset import FlowersDataset, get_test_transforms


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_ds = FlowersDataset("../data/processed/test", transform=get_test_transforms())
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)
num_classes = 5


In [ ]:
def load_custom_cnn(path):
    model = CustomCNN(num_classes=num_classes).to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    return model

def load_resnet18_fe(path):
    model = get_resnet18(num_classes=num_classes, feature_extract=True).to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    return model

def load_resnet18_ft(path):
    model = get_resnet18(num_classes=num_classes, feature_extract=False).to(device)
    model.load_state_dict(torch.load(path, map_location=device))
    return model

In [ ]:
results = []

# --------------------------
# 1. Custom CNN
# --------------------------
custom_path = "../checkpoints/baseline_best.pth"
custom_model = load_custom_cnn(custom_path)
custom_val_acc = 0.62            # те, що отримав під час тренування
custom_test_acc = evaluate(custom_model, test_loader, device)
custom_size = model_size(custom_path)
custom_time = 12                 # хвилини (постав свої дані)

results.append([
    "Custom CNN",
    custom_val_acc,
    custom_test_acc,
    custom_size,
    custom_time,
    "Власна модель"
])


# --------------------------
# 2. ResNet18 FE
# --------------------------
fe_path = "../checkpoints/resnet_fe.pth"
resnet_fe = load_resnet18_fe(fe_path)
resnet_fe_val_acc = 0.8292       # приклад (замінити)
resnet_fe_test_acc = evaluate(resnet_fe, test_loader, device)
resnet_fe_size = model_size(fe_path)
resnet_fe_time = 3.3

results.append([
    "ResNet18 - Feature Extraction",
    resnet_fe_val_acc,
    resnet_fe_test_acc,
    resnet_fe_size,
    resnet_fe_time,
    "Заморожений backbone"
])


# --------------------------
# 3. ResNet18 FT
# --------------------------
ft_path = "../checkpoints/resnet_ft.pth"
resnet_ft = load_resnet18_ft(ft_path)
resnet_ft_val_acc = 0.882
resnet_ft_test_acc = evaluate(resnet_ft, test_loader, device)
resnet_ft_size = model_size(ft_path)
resnet_ft_time = 12

results.append([
    "ResNet18 - Fine Tuning",
    resnet_ft_val_acc,
    resnet_ft_test_acc,
    resnet_ft_size,
    resnet_ft_time,
    "Розморожені шари"
])


# --------------------------
# 4. FT + Baseline Augmentation
# --------------------------
ft_aug_path = "../checkpoints/resnet_baseline_aug.pth"
resnet_ft_aug = load_resnet18_ft(ft_aug_path)
resnet_ft_aug_val_acc = 0.901
resnet_ft_aug_test_acc = evaluate(resnet_ft_aug, test_loader, device)
resnet_ft_aug_size = model_size(ft_aug_path)
resnet_ft_aug_time = 13

results.append([
    "ResNet18 - FT + Baseline Aug",
    resnet_ft_aug_val_acc,
    resnet_ft_aug_test_acc,
    resnet_ft_aug_size,
    resnet_ft_aug_time,
    "Baseline Augmentations"
])


# --------------------------
# 5. FT + Advanced Aug
# --------------------------
adv_aug_path = "../checkpoints/resnet_advanced_aug.pth"
resnet_adv = load_resnet18_ft(adv_aug_path)
resnet_adv_val_acc = 0.918
resnet_adv_test_acc = evaluate(resnet_adv, test_loader, device)
resnet_adv_size = model_size(adv_aug_path)
resnet_adv_time = 14

results.append([
    "ResNet18 - FT + Advanced Aug",
    resnet_adv_val_acc,
    resnet_adv_test_acc,
    resnet_adv_size,
    resnet_adv_time,
    "Advanced Augmentations"
])


# --------------------------
# 6. FT + Advanced Aug + Reg (BEST)
# --------------------------
best_path = "../checkpoints/resnet_best.pth"
resnet_best = load_resnet18_ft(best_path)
resnet_best_val_acc = 0.934
resnet_best_test_acc = evaluate(resnet_best, test_loader, device)
resnet_best_size = model_size(best_path)
resnet_best_time = 16

results.append([
    "ResNet18 - FT + Aug + Regularization",
    resnet_best_val_acc,
    resnet_best_test_acc,
    resnet_best_size,
    resnet_best_time,
    "Dropout + L2 + Label Smoothing"
])

In [ ]:
df = pd.DataFrame(
    results,
    columns=[
        "Model",
        "Val Accuracy",
        "Test Accuracy",
        "Model Size (MB)",
        "Train Time (min)",
        "Notes"
    ]
)

display(df)

In [ ]:
df.to_csv("../results/final_report_table.csv", index=False)
df.to_markdown("../results/final_report_table.md", index=False)